In [1]:
pip install mediapipe


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install mediapipe


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.1.2 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip show mediapipe


Name: mediapipe
Version: 0.10.14
Summary: MediaPipe is the simplest way for researchers and developers to build world-class ML solutions and applications for mobile, edge, cloud and the web.
Home-page: https://github.com/google/mediapipe
Author: The MediaPipe Authors
Author-email: mediapipe@google.com
License: Apache 2.0
Location: c:\users\moses\appdata\roaming\python\python39\site-packages
Requires: absl-py, attrs, flatbuffers, jax, jaxlib, matplotlib, numpy, opencv-contrib-python, protobuf, sounddevice
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Helper function to calculate angle between three points
def calculate_angle(a, b, c):
    """Calculate the angle between three points in degrees."""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0])
    angle = abs(math.degrees(radians))
    if angle > 180.0:
        angle = 360 - angle
    return angle

def classify_exercise(landmarks):
    """Classify exercise type based on landmark angles."""
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

    # Calculate angles for exercise classification
    knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    return knee_angle, elbow_angle

def count_reps(exercise, knee_angle, elbow_angle, prev_state, exercise_counts):
    """Count reps based on exercise type and angle changes."""
    current_state = prev_state
    rep_count = 0
    new_state = None  # Initialize new_state

    # Detecting state transitions based on exercise type
    if exercise == 'squat':
        new_state = 'down' if knee_angle < 90 else 'up'
        print(f"Squat - Knee Angle: {knee_angle}, State: {new_state}")
        if prev_state == 'down' and new_state == 'up':
            rep_count = 1
    elif exercise == 'push_up':
        new_state = 'down' if elbow_angle < 30 else 'up'
        print(f"Push-Up - Elbow Angle: {elbow_angle}, State: {new_state}")
        if prev_state == 'down' and new_state == 'up':
            rep_count = 1
    elif exercise == 'burpee':
        if knee_angle < 90:
            new_state = 'squat_down'
        elif elbow_angle < 30:
            new_state = 'push_down'
        elif prev_state == 'push_down' and knee_angle > 170:
            new_state = 'jump'
            rep_count = 1
        print(f"Burpee - Knee Angle: {knee_angle}, Elbow Angle: {elbow_angle}, State: {new_state}")

    current_state = new_state

    # Update counts based on current exercise
    if rep_count:
        exercise_counts[exercise] += rep_count
        print(f"Exercise: {exercise}, Reps: {exercise_counts[exercise]}")

    return current_state, exercise_counts

def process_video(video_source=0):
    """Process video for exercise detection and display results on screen."""
    cap = cv2.VideoCapture(video_source)
    exercise_counts = {'push_up': 0, 'squat': 0, 'burpee': 0}
    prev_state = {'squat': 'up', 'push_up': 'up', 'burpee': 'up'}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize the frame
        frame = cv2.resize(frame, (640, 480))
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame)
        current_exercise = None
        if results.pose_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            knee_angle, elbow_angle = classify_exercise(results.pose_landmarks.landmark)

            if knee_angle < 90:
                current_exercise = 'squat'
            elif elbow_angle < 30:
                current_exercise = 'push_up'
            else:
                current_exercise = 'burpee'

            prev_state[current_exercise], exercise_counts = count_reps(current_exercise, knee_angle, elbow_angle, prev_state[current_exercise], exercise_counts)

        display_text = f"Push-ups: {exercise_counts['push_up']} | Squats: {exercise_counts['squat']} | Burpees: {exercise_counts['burpee']}"
        cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Exercise Detection', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return exercise_counts

def summarize_log(exercise_counts):
    """Summarize each exercise and total repetitions."""
    for idx, (exercise, reps) in enumerate(exercise_counts.items(), 1):
        print(f"Exercise-{idx} : {exercise}")
        print(f"Reps: {reps}\n")

def main():
    print("Choose an option: 1 for Live Detection, 2 for Video File")
    choice = input("Enter 1 or 2: ")
    if choice == '1':
        exercise_counts = process_video(0)
    elif choice == '2':
        video_path = input("Enter the path to the video file: ")
        exercise_counts = process_video(video_path)
    else:
        print("Invalid choice. Please enter '1' or '2'.")
        return
    summarize_log(exercise_counts)

if __name__ == "__main__":
    main()


Choose an option: 1 for Live Detection, 2 for Video File
Enter 1 or 2: 2
Enter the path to the video file: C:\Users\MOSES\Downloads\pushUP.mp4
Burpee - Knee Angle: 179.66039123953593, Elbow Angle: 174.59460504865382, State: None
Burpee - Knee Angle: 178.8844107487933, Elbow Angle: 174.83692561599358, State: None
Burpee - Knee Angle: 177.90800111459816, Elbow Angle: 175.02436524711547, State: None
Burpee - Knee Angle: 177.26481994788315, Elbow Angle: 175.1114375559532, State: None
Burpee - Knee Angle: 177.23072855094688, Elbow Angle: 175.1761838906604, State: None
Burpee - Knee Angle: 176.9700720210923, Elbow Angle: 175.18330673785712, State: None
Burpee - Knee Angle: 174.45793621132398, Elbow Angle: 175.23897612420163, State: None
Burpee - Knee Angle: 172.98867157224046, Elbow Angle: 175.2920403936101, State: None
Burpee - Knee Angle: 171.89285899952185, Elbow Angle: 175.27347150795816, State: None
Burpee - Knee Angle: 173.35994283524553, Elbow Angle: 175.25572374101066, State: None
Bu

Burpee - Knee Angle: 174.74256486990103, Elbow Angle: 178.32908154643832, State: None
Burpee - Knee Angle: 174.26848825537644, Elbow Angle: 176.4535676844291, State: None
Burpee - Knee Angle: 174.3721078680421, Elbow Angle: 175.69858532353572, State: None
Burpee - Knee Angle: 175.2720536156579, Elbow Angle: 175.46500287865558, State: None
Burpee - Knee Angle: 174.78223080067448, Elbow Angle: 175.44305623364409, State: None
Burpee - Knee Angle: 174.4581443885962, Elbow Angle: 174.81916003971588, State: None
Burpee - Knee Angle: 174.82445241237886, Elbow Angle: 174.61372042460292, State: None
Burpee - Knee Angle: 175.5300388756166, Elbow Angle: 173.1951228970372, State: None
Burpee - Knee Angle: 176.76010967419407, Elbow Angle: 168.893415813495, State: None
Burpee - Knee Angle: 177.81783987034765, Elbow Angle: 165.45339790605755, State: None
Burpee - Knee Angle: 176.45101158227237, Elbow Angle: 162.1824900064847, State: None
Burpee - Knee Angle: 167.86902133022036, Elbow Angle: 157.76948

Burpee - Knee Angle: 166.46461400254188, Elbow Angle: 144.96227812969465, State: None
Burpee - Knee Angle: 168.89280740164634, Elbow Angle: 140.4477339314087, State: None
Burpee - Knee Angle: 172.1513399045877, Elbow Angle: 134.77545923999216, State: None
Burpee - Knee Angle: 169.2672309800021, Elbow Angle: 127.6283470106772, State: None
Burpee - Knee Angle: 171.2818272622383, Elbow Angle: 121.57614496218602, State: None
Burpee - Knee Angle: 172.56673014493603, Elbow Angle: 115.91974160312796, State: None
Burpee - Knee Angle: 172.92184172402582, Elbow Angle: 109.45913379240812, State: None
Burpee - Knee Angle: 173.03372766093034, Elbow Angle: 100.88558311297827, State: None
Burpee - Knee Angle: 172.81381543298156, Elbow Angle: 94.47971165825827, State: None
Burpee - Knee Angle: 172.09829262025644, Elbow Angle: 82.37240312743482, State: None
Burpee - Knee Angle: 171.65363339529515, Elbow Angle: 65.02767309197465, State: None
Burpee - Knee Angle: 171.0469100208501, Elbow Angle: 55.983621

Burpee - Knee Angle: 158.9876380337632, Elbow Angle: 148.5670503462835, State: None
Burpee - Knee Angle: 159.50238315752642, Elbow Angle: 163.85930846549752, State: None
Burpee - Knee Angle: 161.03789434656935, Elbow Angle: 168.42207165729988, State: None
Burpee - Knee Angle: 160.7381591467063, Elbow Angle: 170.74819405788523, State: None
Burpee - Knee Angle: 161.23200520802362, Elbow Angle: 174.69854361429864, State: None
Burpee - Knee Angle: 164.79755169400408, Elbow Angle: 173.82043567500355, State: None
Burpee - Knee Angle: 167.4159667124469, Elbow Angle: 174.1181085451419, State: None
Burpee - Knee Angle: 168.318834914018, Elbow Angle: 172.4460819643047, State: None
Burpee - Knee Angle: 172.4758289848693, Elbow Angle: 172.64225851625963, State: None
Burpee - Knee Angle: 174.65227140427854, Elbow Angle: 175.0689038815599, State: None
Burpee - Knee Angle: 174.17449606513267, Elbow Angle: 174.5923143120517, State: None
Burpee - Knee Angle: 174.12382835353145, Elbow Angle: 174.2998392